# Finetune `meta-llama/Meta-Llama-3-8B-Instruct` on an EC2 instance using `Unsloth`
---

Unsloth makes finetuning large language models like Llama-3, Mistral, Phi-4 and Gemma 2x faster, use 70% less memory, and with no degradation in accuracy!

**Note**: ***This notebook is run on a `g6e.12xlarge` instance. Follow the prerequisite steps [here](README.md)***

In this example, we will be fine tuning the llama3 8b instruct model. There are several 4bit pre quantized models that `unsloth` provides that are not gated. This supports 4x faster downloading with no OOMs. In this case, we will be using the standard `meta-llama/Meta-Llama-3-8B-Instruct` model from hugging face. 

In [1]:
import os
import logging
import globals as g
from dotenv import load_dotenv
from unsloth import to_sharegpt
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import standardize_sharegpt
from ec2_metrics import EC2MetricsCallback

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1891: SyntaxWarning: invalid escape sequence '\.'
  start = re.search('logger\.info\([\"\'].+?Running training', inner_training_loop).span(0)[0]
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1894: SyntaxWarning: invalid escape sequence '\s'
  spaces = re.search('\n([\s\t]{1,})', original_debug).group(0)[1:]
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1895: SyntaxWarning: invalid escape sequence '\s'
  front_spaces = re.match('([\s\t]{1,})', inner_training_loop).group(0)
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/models/_utils.py:592: SyntaxWarning: invalid escape sequence '\.'
  source = re.sub("([^\.])nn\.", r"\1torch.nn.", source)
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/py

🦥 Unsloth Zoo will now patch everything to make training faster!


/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/tokenizer_utils.py:962: SyntaxWarning: invalid escape sequence '\('
  f"def {function_name}\(.*?\).*?\:\n",
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/save.py:485: SyntaxWarning: invalid escape sequence '\s'
  gb_found = re.match("([0-9]{1,})[\s]{0,}GB", max_shard_size, flags = re.IGNORECASE)
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/save.py:486: SyntaxWarning: invalid escape sequence '\s'
  mb_found = re.match("([0-9]{1,})[\s]{0,}MB", max_shard_size, flags = re.IGNORECASE)
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/save.py:1020: SyntaxWarning: invalid escape sequence '\ '
  f"   \\\   /|    [0] Installing llama.cpp might take 3 minutes.\n"\
/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/unsloth/sav

In [2]:
# Load environment variables from .env file
import getpass
load_dotenv()
if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your HuggingFace token: ")
hf_token = os.getenv("HF_TOKEN")

if not os.getenv("HF_MODEL_ID"):
    hf_model_id  = input("Enter the model id to use for fine-tuning (e.g. meta-llama/Llama-3.1-8B-Instruct): ")
else:
    hf_model_id = os.getenv("HF_MODEL_ID")
logger.info(f"hf_model_id={hf_model_id}")


[2025-03-15 18:25:08,387] p6428 {2478216038.py:12} INFO - hf_model_id=meta-llama/Llama-3.1-8B-Instruct


In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

DATASET_OF_INTEREST: str = 'vicgalle/alpaca-gpt4'

ALPACA_PROMPT: str = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [4]:
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = hf_model_id,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = hf_token # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
except Exception as e:
    logger.error(f"Error occurred while loading the model: {e}")
    raise

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [02:12<00:00, 33.18s/it]


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

We now use the Alpaca dataset from vicgalle, which is a version of 52K of the original Alpaca dataset generated from GPT4. You can replace this code section with your own data prep.

In [6]:
dataset = load_dataset(DATASET_OF_INTEREST, split="train")
logger.info(f"Columns in the dataset: {dataset.column_names}")

Generating train split: 100%|██████████| 52002/52002 [00:00<00:00, 153306.91 examples/s]
[2025-03-15 18:29:21,974] p6428 {2022229014.py:2} INFO - Columns in the dataset: ['instruction', 'input', 'output', 'text']


In [7]:
dataset = to_sharegpt(
    dataset,
    merged_prompt="{instruction}[[\nYour input is:\n{input}]]",
    output_column_name="output",
    conversation_extension=3,
)

Extending conversations: 100%|██████████| 52002/52002 [00:02<00:00, 21477.78 examples/s]


In [8]:
# Use the standardize_sharegpt function to just make the dataset in a correct format for finetuning
dataset = standardize_sharegpt(dataset)

Standardizing format: 100%|██████████| 52002/52002 [00:02<00:00, 25792.66 examples/s]


In [9]:
from pprint import pprint
pprint(dataset[:3])

{'conversations': [[{'content': 'Give three tips for staying healthy.',
                     'role': 'user'},
                    {'content': '1. Eat a balanced and nutritious diet: Make '
                                'sure your meals are inclusive of a variety of '
                                'fruits and vegetables, lean protein, whole '
                                'grains, and healthy fats. This helps to '
                                'provide your body with the essential '
                                'nutrients to function at its best and can '
                                'help prevent chronic diseases.\n'
                                '\n'
                                '2. Engage in regular physical activity: '
                                'Exercise is crucial for maintaining strong '
                                'bones, muscles, and cardiovascular health. '
                                'Aim for at least 150 minutes of moderate '
                 

In [10]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 52002/52002 [00:04<00:00, 12543.92 examples/s]


In [11]:
%%time
# train the model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    callbacks=[EC2MetricsCallback],
)

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 52002/52002 [00:03<00:00, 14348.26 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 52002/52002 [00:03<00:00, 13157.07 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 52002/52002 [00:13<00:00, 3971.17 examples/s]

CPU times: user 1.5 s, sys: 483 ms, total: 1.98 s
Wall time: 53.3 s


In [ ]:
%%time
# this will initiate the training process and also log the EC2 utilization metrics, such as the GPU
# utilization, CPU utilization, etc.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
[2025-03-15 18:30:34,954] p6428 {ec2_metrics.py:184} INFO - Training started. Initiating EC2 metrics collection.
[2025-03-15 18:30:34,956] p6428 {ec2_metrics.py:170} INFO - Writing header: ['timestamp', 'cpu_percent_mean', 'memory_percent_mean', 'memory_used_mean', 'gpu_utilization_mean', 'gpu_memory_used_mean', 'gpu_memory_free_mean', 'gpu_memory_total_mean']
[2025-03-15 18:30:34,957] p6428 {ec2_metrics.py:41} INFO - Starting collection
[2025-03-15 18:30:35,355] p6428 {ec2_metrics.py:143} INFO - Starting daemon collector to run in background


Step,Training Loss
1,1.446100
2,1.435900
3,1.391000
4,1.485100
5,1.569200
6,1.311400
7,1.335300
8,1.326000
9,1.342800
10,1.229500


[2025-03-15 18:36:33,427] p6428 {ec2_metrics.py:191} INFO - Training ended. Stopping EC2 metrics collection.
[2025-03-15 18:36:33,428] p6428 {ec2_metrics.py:33} INFO - Stopped collection


CPU times: user 3min 22s, sys: 2min 32s, total: 5min 55s
Wall time: 5min 59s


[2025-03-15 18:36:35,427] p6428 {ec2_metrics.py:33} INFO - Stopped collection


### Log the trainer stats
---

In this step, we log some of the trainer stats, such as the number of global steps it took to get to a specific training loss, the train runtime, samples per second, steps per second, etc.

In [13]:
# Format the training stats in a readable way
output_text = f"""Training Statistics:
Global Steps: {trainer_stats.global_step}
Training Loss: {trainer_stats.training_loss:.4f}

Metrics:
- Train Runtime: {trainer_stats.metrics['train_runtime']:.3f} seconds
- Training Samples/Second: {trainer_stats.metrics['train_samples_per_second']:.3f}
- Training Steps/Second: {trainer_stats.metrics['train_steps_per_second']:.3f}
- Total FLOPS: {trainer_stats.metrics['total_flos']:.2e}
- Final Train Loss: {trainer_stats.metrics['train_loss']:.4f}
"""

# Save to a text file
with open(os.path.join(g.RESULTS_DIR, g.TRAINING_STATS), 'w') as f:
    f.write(output_text)

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The next number in the Fibonacci sequence is 13.<|eot_id|>


In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The tallest tower in France is called the Eiffel Tower.<|eot_id|>


In [16]:
# save the model
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

## Ollama Support [Optional]

Unsloth now allows you to automatically finetune and create a Modelfile, and export to Ollama! This makes finetuning much easier and provides a seamless workflow from Unsloth to Ollama!

Let's first install Ollama!

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#########                                                    31.0%##########################################################           88.9%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
>>> NVIDIA GPU installed.


In [18]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


make: Entering directory '/home/ubuntu/sk2338/spring-2025-lab07-briansoonwookwon/llama.cpp'
make: Leaving directory '/home/ubuntu/sk2338/spring-2025-lab07-briansoonwookwon/llama.cpp'


Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


...

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /home/ubuntu/sk2338/spring-2025-lab07-briansoonwookwon/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loadin

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /home/ubuntu/sk2338/spring-2025-lab07-briansoonwookwon/model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


In [19]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3) 

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [20]:
!ollama create unsloth_model -f ./model/Modelfile

gathering model components ⠙ gathering model components ⠙ gathering model components ⠹ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠙ gathering model components ⠹ gathering model components ⠼ gathering model components ⠼ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠹ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering mode

In [ ]:
# run inference against the model
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

In [ ]:
# run inference against the model
import json
result = subprocess.run(
    [
        "curl",
        "http://localhost:11434/api/generate",
        "-d",
        '{"model": "unsloth_model", "prompt": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8,", "stream": false}',
    ],
    capture_output=True,
    text=True,
)

response_data = json.loads(result.stdout)
print(f"Response generated: {response_data['response']}")

